<a href="https://colab.research.google.com/github/BxMild/bsc_dpdm24/blob/main/Pollution_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Prepare

https://data.go.th/dataset/dataset_51_011

In [41]:
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [42]:
df = pd.read_excel('/content/drive/MyDrive/Data_BSC_DPDM24/pollutionpointandreport2565.xlsx')
df

,Province,Quantity,Percentage,Odor,Noise,Dust/Smoke,Water Pollution,Solid waste,Hazardous waste,Vibration Pollution,Others,Total
0,กระบี่,2,0.25,2,0,1,0,0,0,0,0,3
1,กรุงเทพมหานคร,211,26.44,129,91,80,27,10,0,17,19,373
2,กาญจนบุรี,11,1.38,10,0,7,1,1,0,0,0,19
3,กาฬสินธุ์,1,0.13,1,0,0,0,0,0,0,0,1
4,กำแพงเพชร,4,0.50,3,0,3,1,0,0,0,0,7
...,...,...,...,...,...,...,...,...,...,...,...,...
73,อุดรธานี,6,0.75,4,2,3,1,0,0,1,0,11
74,อุตรดิตถ์,5,0.63,1,3,2,1,0,0,0,0,7
75,อุทัยธานี,5,0.63,3,1,5,0,0,0,0,0,9
76,อุบลราชธานี,9,1.13,8,0,5,4,0,0,0,1,18


In [43]:
df = df[df['Province'] != 'รวม']
df

,Province,Quantity,Percentage,Odor,Noise,Dust/Smoke,Water Pollution,Solid waste,Hazardous waste,Vibration Pollution,Others,Total
0,กระบี่,2,0.25,2,0,1,0,0,0,0,0,3
1,กรุงเทพมหานคร,211,26.44,129,91,80,27,10,0,17,19,373
2,กาญจนบุรี,11,1.38,10,0,7,1,1,0,0,0,19
3,กาฬสินธุ์,1,0.13,1,0,0,0,0,0,0,0,1
4,กำแพงเพชร,4,0.50,3,0,3,1,0,0,0,0,7
...,...,...,...,...,...,...,...,...,...,...,...,...
72,อำนาจเจริญ,0,0.00,0,0,0,0,0,0,0,0,0
73,อุดรธานี,6,0.75,4,2,3,1,0,0,1,0,11
74,อุตรดิตถ์,5,0.63,1,3,2,1,0,0,0,0,7
75,อุทัยธานี,5,0.63,3,1,5,0,0,0,0,0,9


In [44]:
print(df.isnull().sum())

Province               0
Quantity               0
Percentage             0
Odor                   0
Noise                  0
Dust/Smoke             0
Water Pollution        0
Solid waste            0
Hazardous waste        0
Vibration Pollution    0
Others                 0
Total                  0
dtype: int64


In [45]:
print(df.dtypes)

Province                object
Quantity                 int64
Percentage             float64
Odor                     int64
Noise                    int64
Dust/Smoke               int64
Water Pollution          int64
Solid waste              int64
Hazardous waste          int64
Vibration Pollution      int64
Others                   int64
Total                    int64
dtype: object


# Association Rulles

In [46]:
df.columns

Index(['Province', 'Quantity', 'Percentage', 'Odor', 'Noise', 'Dust/Smoke',
       'Water Pollution', 'Solid waste', 'Hazardous waste',
       'Vibration Pollution', 'Others', 'Total'],
      dtype='object')

In [47]:
pollution = df[['Province', 'Odor', 'Noise', 'Dust/Smoke',
       'Water Pollution', 'Solid waste', 'Hazardous waste',
       'Vibration Pollution', 'Others']]

## โหลดข้อมูลและแปลงให้เป็น One-Hot Encoding

In [48]:
# แปลงข้อมูลเป็น One-Hot Encoding โดยกำหนดว่า ถ้ามีมลพิษประเภทใดในจังหวัดนั้น ให้เป็น 1
df_encoded = pollution.drop('Province', axis=1).apply(lambda x: x > 0, axis=1).astype(int)

# ดูข้อมูลที่แปลงแล้ว
df_encoded.head()

,Odor,Noise,Dust/Smoke,Water Pollution,Solid waste,Hazardous waste,Vibration Pollution,Others
0,1,0,1,0,0,0,0,0
1,1,1,1,1,1,0,1,1
2,1,0,1,1,1,0,0,0
3,1,0,0,0,0,0,0,0
4,1,0,1,1,0,0,0,0


## ใช้ Apriori Algorithm เพื่อหากฎ Association

In [49]:
from mlxtend.frequent_patterns import apriori, association_rules

# ใช้ Apriori Algorithm เพื่อหากฎ
frequent_itemsets = apriori(df_encoded, min_support=0.1, use_colnames=True)

# สร้าง Association Rules
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)

# ดูกฎที่พบ
print(rules)


           antecedents                                   consequents  \
0               (Odor)                                       (Noise)   
1              (Noise)                                        (Odor)   
2               (Odor)                                  (Dust/Smoke)   
3         (Dust/Smoke)                                        (Odor)   
4               (Odor)                             (Water Pollution)   
..                 ...                                           ...   
435             (Odor)  (Dust/Smoke, Others, Water Pollution, Noise)   
436       (Dust/Smoke)        (Odor, Others, Water Pollution, Noise)   
437           (Others)    (Odor, Dust/Smoke, Water Pollution, Noise)   
438  (Water Pollution)             (Odor, Dust/Smoke, Others, Noise)   
439            (Noise)   (Odor, Dust/Smoke, Others, Water Pollution)   

     antecedent support  consequent support   support  confidence      lift  \
0              0.779221            0.558442  0.519481   

/usr/local/lib/python3.11/dist-packages/mlxtend/frequent_patterns/fpcommon.py:161: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


In [50]:
# สรุปผลกฎที่ได้จาก Association Rules
# กรองกฎที่มีค่า lift สูงสุดและน่าสนใจ
rules_sorted = rules.sort_values(by="lift", ascending=False)

# แสดง 10 กฎที่มีค่า lift สูงสุด
top_rules = rules_sorted.head(10)

# แสดงกฎที่ได้
print("Top 10 Association Rules based on Lift:")
print(top_rules)

# แสดงข้อมูลสำคัญของแต่ละกฎ
for index, row in top_rules.iterrows():
    print(f"Rule {index + 1}:")
    print(f"  - Antecedents: {row['antecedents']}")
    print(f"  - Consequents: {row['consequents']}")
    print(f"  - Support: {row['support']}")
    print(f"  - Confidence: {row['confidence']}")
    print(f"  - Lift: {row['lift']}")
    print("-" * 50)

Top 10 Association Rules based on Lift:
                                    antecedents  \
332                       (Vibration Pollution)   
325        (Dust/Smoke, Water Pollution, Noise)   
392        (Dust/Smoke, Water Pollution, Noise)   
381  (Odor, Dust/Smoke, Water Pollution, Noise)   
408                       (Vibration Pollution)   
397                 (Odor, Vibration Pollution)   
242                 (Odor, Vibration Pollution)   
247                    (Water Pollution, Noise)   
240              (Odor, Water Pollution, Noise)   
326           (Vibration Pollution, Dust/Smoke)   

                                    consequents  antecedent support  \
332        (Dust/Smoke, Water Pollution, Noise)            0.181818   
325                       (Vibration Pollution)            0.441558   
392                 (Odor, Vibration Pollution)            0.441558   
381                       (Vibration Pollution)            0.441558   
408  (Odor, Dust/Smoke, Water Pollution, No

* กฎที่ 332:

  -  Antecedents: มลพิษจากการสั่นสะเทือน
  - Consequents: ฝุ่น/ควัน, มลพิษน้ำ, เสียง
  - Lift: 2.26
  - การตีความ:
มลพิษจากการสั่นสะเทือนมักจะเชื่อมโยงกับการเกิดพร้อมกันของฝุ่น/ควัน, มลพิษน้ำ, และเสียง โดยมีความมั่นใจสูง (1.0) ซึ่งหมายความว่าเมื่อมลพิษจากการสั่นสะเทือนเกิดขึ้น มลพิษประเภทอื่นๆ เหล่านี้มักจะเกิดตามมาเกือบทุกครั้ง

* กฎที่ 325:

Antecedents: ฝุ่น/ควัน, มลพิษน้ำ, เสียง
Consequents: มลพิษจากการสั่นสะเทือน
Lift: 2.26
การตีความ: ฝุ่น/ควัน, มลพิษน้ำ, และเสียงมักจะเกิดก่อนมลพิษจากการสั่นสะเทือน โดยมีความมั่นใจที่ต่ำกว่า (0.41) แสดงว่ามลพิษเหล่านี้สามารถนำไปสู่การเกิดมลพิษจากการสั่นสะเทือนได้ แต่ไม่เกิดขึ้นทุกครั้ง

* กฎที่ 392:

Antecedents: ฝุ่น/ควัน, มลพิษน้ำ, เสียง
Consequents: กลิ่น, มลพิษจากการสั่นสะเทือน
Lift: 2.26
การตีความ: ฝุ่น/ควัน, มลพิษน้ำ, และเสียงสามารถนำไปสู่การเกิดกลิ่นและมลพิษจากการสั่นสะเทือนได้ โดยมีความมั่นใจที่ 0.41

* กฎที่ 381:

Antecedents: กลิ่น, ฝุ่น/ควัน, มลพิษน้ำ, เสียง
Consequents: มลพิษจากการสั่นสะเทือน
Lift: 2.26
การตีความ: เมื่อมลพิษหลายประเภทเกิดขึ้นร่วมกัน เช่น กลิ่น, ฝุ่น/ควัน, มลพิษน้ำ, และเสียง มลพิษจากการสั่นสะเทือนมักจะตามมา โดยมีความมั่นใจที่ 0.41

* กฎที่ 408:

Antecedents: มลพิษจากการสั่นสะเทือน
Consequents: กลิ่น, ฝุ่น/ควัน, มลพิษน้ำ, เสียง
Lift: 2.26
การตีความ: มลพิษจากการสั่นสะเทือนมักจะเชื่อมโยงกับการเกิดกลิ่น, ฝุ่น/ควัน, มลพิษน้ำ, และเสียง โดยมีความมั่นใจสูง (1.0)

# Model

In [51]:
# นำเข้าข้อมูล
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score


In [52]:
data = df.copy()

In [54]:
# นำเข้าข้อมูล
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score

data = df.copy()

# Define the thresholds
low_threshold = 10  # Example value
high_threshold = 30 # Example value

# สร้างฟังก์ชันเพื่อแบ่งระดับมลพิษ
def classify_pollution(total_pollution):
    if total_pollution < low_threshold:
        return 'Low'
    elif low_threshold <= total_pollution < high_threshold:
        return 'Medium'
    else:
        return 'High'

data['Pollution_Severity'] = data['Total'].apply(classify_pollution)

In [57]:
data.columns

Index(['Province', 'Quantity', 'Percentage', 'Odor', 'Noise', 'Dust/Smoke',
       'Water Pollution', 'Solid waste', 'Hazardous waste',
       'Vibration Pollution', 'Others', 'Total', 'Pollution_Severity'],
      dtype='object')

## แบ่งข้อมูล

In [60]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# แบ่งข้อมูล
X = data[['Odor', 'Noise', 'Dust/Smoke', 'Water Pollution', 'Solid waste',
          'Hazardous waste', 'Vibration Pollution', 'Others']]
y = data['Pollution_Severity']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## 1. Logistic Regression

In [62]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score

# สร้างโมเดล Logistic Regression
logreg = LogisticRegression(max_iter=1000)

# ฝึกโมเดล
logreg.fit(X_train, y_train)

# ทำนายผล
y_pred_logreg = logreg.predict(X_test)

# ประเมินผล
print("Logistic Regression - Accuracy:", accuracy_score(y_test, y_pred_logreg))
print("Logistic Regression - Classification Report:\n", classification_report(y_test, y_pred_logreg))


Logistic Regression - Accuracy: 0.875
Logistic Regression - Classification Report:
               precision    recall  f1-score   support

        High       1.00      0.50      0.67         4
         Low       1.00      1.00      1.00        10
      Medium       0.50      1.00      0.67         2

    accuracy                           0.88        16
   macro avg       0.83      0.83      0.78        16
weighted avg       0.94      0.88      0.88        16



## 2. Random Forest

In [63]:
from sklearn.ensemble import RandomForestClassifier

# สร้างโมเดล Random Forest
rf = RandomForestClassifier()

# ฝึกโมเดล
rf.fit(X_train, y_train)

# ทำนายผล
y_pred_rf = rf.predict(X_test)

# ประเมินผล
print("Random Forest - Accuracy:", accuracy_score(y_test, y_pred_rf))
print("Random Forest - Classification Report:\n", classification_report(y_test, y_pred_rf))

Random Forest - Accuracy: 0.8125
Random Forest - Classification Report:
               precision    recall  f1-score   support

        High       1.00      0.25      0.40         4
         Low       1.00      1.00      1.00        10
      Medium       0.40      1.00      0.57         2

    accuracy                           0.81        16
   macro avg       0.80      0.75      0.66        16
weighted avg       0.93      0.81      0.80        16



## 3. Support Vector Machine (SVM)

In [64]:
from sklearn.svm import SVC

# สร้างโมเดล Support Vector Machine
svm = SVC()

# ฝึกโมเดล
svm.fit(X_train, y_train)

# ทำนายผล
y_pred_svm = svm.predict(X_test)

# ประเมินผล
print("SVM - Accuracy:", accuracy_score(y_test, y_pred_svm))
print("SVM - Classification Report:\n", classification_report(y_test, y_pred_svm))


SVM - Accuracy: 0.875
SVM - Classification Report:
               precision    recall  f1-score   support

        High       1.00      0.50      0.67         4
         Low       1.00      1.00      1.00        10
      Medium       0.50      1.00      0.67         2

    accuracy                           0.88        16
   macro avg       0.83      0.83      0.78        16
weighted avg       0.94      0.88      0.88        16



4. K-Nearest Neighbors (KNN)

In [65]:
from sklearn.neighbors import KNeighborsClassifier

# สร้างโมเดล K-Nearest Neighbors
knn = KNeighborsClassifier()

# ฝึกโมเดล
knn.fit(X_train, y_train)

# ทำนายผล
y_pred_knn = knn.predict(X_test)

# ประเมินผล
print("KNN - Accuracy:", accuracy_score(y_test, y_pred_knn))
print("KNN - Classification Report:\n", classification_report(y_test, y_pred_knn))


KNN - Accuracy: 0.875
KNN - Classification Report:
               precision    recall  f1-score   support

        High       1.00      0.50      0.67         4
         Low       1.00      1.00      1.00        10
      Medium       0.50      1.00      0.67         2

    accuracy                           0.88        16
   macro avg       0.83      0.83      0.78        16
weighted avg       0.94      0.88      0.88        16



## 5. Gradient Boosting

In [66]:
from sklearn.ensemble import GradientBoostingClassifier

# สร้างโมเดล Gradient Boosting
gb = GradientBoostingClassifier()

# ฝึกโมเดล
gb.fit(X_train, y_train)

# ทำนายผล
y_pred_gb = gb.predict(X_test)

# ประเมินผล
print("Gradient Boosting - Accuracy:", accuracy_score(y_test, y_pred_gb))
print("Gradient Boosting - Classification Report:\n", classification_report(y_test, y_pred_gb))


Gradient Boosting - Accuracy: 0.8125
Gradient Boosting - Classification Report:
               precision    recall  f1-score   support

        High       1.00      0.25      0.40         4
         Low       1.00      1.00      1.00        10
      Medium       0.40      1.00      0.57         2

    accuracy                           0.81        16
   macro avg       0.80      0.75      0.66        16
weighted avg       0.93      0.81      0.80        16



จากผลการทดสอบโมเดลต่างๆ ที่ได้รับ เราสามารถสรุปผลได้ดังนี้:

1. Logistic Regression
Accuracy: 0.875
Classification Report:
High: Precision = 1.00, Recall = 0.50, F1-score = 0.67
Low: Precision = 1.00, Recall = 1.00, F1-score = 1.00
Medium: Precision = 0.50, Recall = 1.00, F1-score = 0.67
Summary: โมเดลนี้มีความแม่นยำสูง (87.5%) แต่พบว่ามีปัญหาในการทำนายประเภท "High" และ "Medium" เนื่องจากมีจำนวนตัวอย่างน้อย จึงทำให้ค่า Recall และ F1-score ของ "High" และ "Medium" ต่ำ
2. Random Forest
Accuracy: 0.8125
Classification Report:
High: Precision = 1.00, Recall = 0.25, F1-score = 0.40
Low: Precision = 1.00, Recall = 1.00, F1-score = 1.00
Medium: Precision = 0.40, Recall = 1.00, F1-score = 0.57
Summary: โมเดลนี้มีความแม่นยำ 81.25% และแสดงความแม่นยำสูงใน "Low" (1.00 precision และ recall) แต่ไม่สามารถทำนายประเภท "High" ได้ดี (recall ต่ำมาก) และ "Medium" ก็ยังมีผลลัพธ์ไม่ดีนัก
3. SVM
Accuracy: 0.875
Classification Report:
High: Precision = 1.00, Recall = 0.50, F1-score = 0.67
Low: Precision = 1.00, Recall = 1.00, F1-score = 1.00
Medium: Precision = 0.50, Recall = 1.00, F1-score = 0.67
Summary: ผลลัพธ์คล้ายกับ Logistic Regression โดยมีความแม่นยำ 87.5% แต่ก็มีปัญหาคล้ายกันในการทำนาย "High" และ "Medium"
4. KNN
Accuracy: 0.875
Classification Report:
High: Precision = 1.00, Recall = 0.50, F1-score = 0.67
Low: Precision = 1.00, Recall = 1.00, F1-score = 1.00
Medium: Precision = 0.50, Recall = 1.00, F1-score = 0.67
Summary: KNN มีผลลัพธ์คล้ายกับ SVM และ Logistic Regression โดยมีความแม่นยำ 87.5% แต่ไม่สามารถทำนาย "High" และ "Medium" ได้ดี
5. Gradient Boosting
Accuracy: 0.8125
Classification Report:
High: Precision = 1.00, Recall = 0.25, F1-score = 0.40
Low: Precision = 1.00, Recall = 1.00, F1-score = 1.00
Medium: Precision = 0.40, Recall = 1.00, F1-score = 0.57
Summary: Gradient Boosting มีผลลัพธ์คล้ายกับ Random Forest โดยมีความแม่นยำ 81.25% และทำได้ดีในการทำนาย "Low" แต่ไม่สามารถทำนาย "High" ได้ดี

การสรุป:

โมเดลที่มีความแม่นยำสูงที่สุดคือ Logistic Regression, SVM, และ KNN ซึ่งทั้งหมดมีความแม่นยำ 87.5% แต่พบปัญหาในการทำนายประเภท "High" และ "Medium" ที่มีจำนวนตัวอย่างน้อย
Random Forest และ Gradient Boosting มีความแม่นยำต่ำกว่า (ประมาณ 81.25%) และมีปัญหาในการทำนาย "High"
โมเดลทั้งหมดทำได้ดีในการทำนาย "Low" แต่ยังต้องพัฒนาในการทำนายประเภทอื่นๆ โดยเฉพาะ "High" และ "Medium"